In [1]:
import torch
import torch.nn as nn
import math

In [2]:

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        # 创建位置编码矩阵
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 添加一个维度以匹配输入形状 (batch_size, seq_len, d_model)
        self.pe = pe.unsqueeze(0)
    def forward(self, x):
        # 扩展位置编码以匹配批次大小和序列长度
        pe = self.pe[:, :x.size(1), :]
        print(x.shape,pe.shape)
        x = x + pe
        return self.dropout(x)

In [3]:
class TransformerModel(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward, dropout=0.1, vocab_size=10000):
        super(TransformerModel, self).__init__()
        # Transformer的编码器部分
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        # 位置编码
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        # 输入嵌入层
        self.embedding = nn.Embedding(vocab_size, d_model)
    def forward(self, src):
        # 计算位置编码
        src = self.pos_encoder(src)
        # 通过嵌入层
        src = self.embedding(src)
        # 通过Transformer编码器
        output = self.transformer_encoder(src)
        return output

In [4]:
# 参数示例
d_model = 512  # 模型维度
nhead = 8     # 注意力头数
num_layers = 6  # 编码器层数
dim_feedforward = 2048  # 前馈网络的维度
dropout = 0.1  # 丢弃率
vocab_size = 10000  # 词汇表大小

In [5]:
# 创建模型实例
model = TransformerModel(d_model, nhead, num_layers, dim_feedforward, dropout, vocab_size)
model

d:\Anaconda3\envs\game\lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TransformerModel(
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (embedding): Embedding(10000, 512)
)

In [6]:
# 示例输入
src = torch.randint(0, vocab_size, (1,10, 512))  # 10个长度为20的序列
# 前向传播
output = model(src)
print(output.shape)  # 输出形状应为 (10, 20, 512)

torch.Size([1, 10, 512]) torch.Size([1, 10, 512])


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)